In [1]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
from scipp import curve_fit
from scipy.special import voigt_profile
from functools import partial

%matplotlib widget


In [2]:

# filename='../../nano/nano_150K_6.5a_01.q090'
filename='../../nano/nano_1.5K_6.5a_01.q090'
data_array = np.loadtxt(filename)
energy=sc.array(dims=['row'],values=data_array[:, 0],unit='meV')
intensity=sc.array(dims=['row'],values=data_array[:, 1],variances=data_array[:, 2]*data_array[:, 2]) 
data=sc.DataArray(data=intensity, coords={'energy': energy})
data_grouped=data.group('energy').bins.mean()

sc.plot(data_grouped)




InteractiveFig(children=(HBar(), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', layou…

In [3]:
non_nan_mask=sc.isnan(data_grouped.data)

data_grouped.masks['not_nan']=non_nan_mask

In [4]:

## 3. Define resolution function and fit data
def resolution_function_scipp(energy, energy_offset, res_gauss1_area, res_gauss1_sigma, res_BG):
    """
    Calculate the resolution function using the given parameters.

    Parameters:
    energy (array-like): The energy values.
    energy_offset (float): The energy offset.
    res_gauss1_area (float): The area of the first Gaussian component.
    res_gauss1_sigma (float): The standard deviation of the first Gaussian component.
    res_BG (float): The background value.

    Returns:
    array-like: The calculated resolution function values.
    """
    x = energy
    x = x - energy_offset

    y = res_gauss1_area * 1 / np.sqrt(2 * np.pi) / res_gauss1_sigma * sc.exp(-0.5 * (x / res_gauss1_sigma) ** 2) + res_BG
    return y


ENERGY_MIN=-0.4*sc.Unit('meV')
ENERGY_MAX=0.4*sc.Unit('meV')

data_grouped_for_fit=data_grouped['energy',ENERGY_MIN:ENERGY_MAX]

popt, _ = curve_fit(['energy'], resolution_function_scipp, data_grouped_for_fit, 
                        p0 = {'res_gauss1_area': 75.0*sc.Unit('meV'), 
                            'res_gauss1_sigma': 0.02 * sc.Unit('meV'),
                            'energy_offset'   : 0 * sc.Unit('meV'),
                            'res_BG'          :0.05})

popt

DataGroup(sizes={}, keys=[
    res_gauss1_area: DataArray({}),
    energy_offset: DataArray({}),
    res_BG: DataArray({}),
    res_gauss1_sigma: DataArray({}),
])

In [5]:

## TODO: The function should include detailed balance
def hem_BG_fit_function_scipp(energy, energy_offset, res_gauss1_sigma,el_area,lorz1_area,lorz1_HWHM,BG):
    """
    Elastic plus quasielastic signal with a Gaussian resolution and a constant background

    Parameters:
    energy (array-like): The energy values.
    energy_offset (float): The energy offset.
    res_gauss1_sigma (float): The standard deviation of the Gaussian resolution function.
    el_area (float): The area of the elastic peak.
    lorz1_area (float): The area of the Lorentzian peak.
    lorz1_HWHM (float): The half-width at half-maximum of the Lorentzian peak.
    BG (float): The background value.

    Returns:
    array-like: The calculated GGG fit function values.
    """

    # Subtract energy offset from energy values
    x = energy-energy_offset

    # Calculate elastic peak using resolution function
    y_el = resolution_function_scipp(energy, energy_offset, el_area, res_gauss1_sigma, 0)

    # scipp doesn't allow using some functions like the voigt_profile, imported from scipy. We therefore need to convert the scipp arrays to numpy arrays.
    # TODO Make a scipp version of the voigt_profile
    y_lorz=lorz1_area.value*(voigt_profile(x.values, res_gauss1_sigma.value, lorz1_HWHM.value))

    # Sum elastic and Lorentzian peaks and add background
    y = y_el.values + y_lorz + BG.values

    # Return the result as a scipp array
    y=sc.array(dims=['energy'],values=y,unit='dimensionless')    
    
    return y




In [6]:


ENERGY_MIN=-2*sc.Unit('meV')
ENERGY_MAX=2*sc.Unit('meV')


filename='../../nano/nano_150K_6.5a_01.q090'
data_array = np.loadtxt(filename)
energy=sc.array(dims=['row'],values=data_array[:, 0],unit='meV')
intensity=sc.array(dims=['row'],values=data_array[:, 1],variances=data_array[:, 2]*data_array[:, 2]) 
data_150K=sc.DataArray(data=intensity, coords={'energy': energy})
data_grouped_150K=data_150K.group('energy').bins.mean()


non_nan_mask_150K=sc.isnan(data_grouped_150K.data)

data_grouped_150K.masks['not_nan']=non_nan_mask_150K

data_grouped_for_fit_150K=data_grouped_150K['energy',ENERGY_MIN:ENERGY_MAX]
data_grouped_for_fit_150K

<scipp.DataArray>
Dimensions: Sizes[energy:335, ]
Coordinates:
* energy                    float64            [meV]  (energy)  [-1.995, -1.985, ..., 1.335, 1.345]
Data:
                            float64  [dimensionless]  (energy)  [0.0689481, 0.0812114, ..., 0.199239, 0.188911]  [0.00014619, 0.00015072, ..., 0.0037938, 0.00524289]
Masks:
  not_nan                      bool        <no unit>  (energy)  [False, False, ..., False, False]

In [7]:



res_gauss1_sigma_fit = popt['res_gauss1_sigma'].value*sc.Unit('meV')

hem_fit_function_fixed_resolution_scipp = partial(hem_BG_fit_function_scipp, res_gauss1_sigma=res_gauss1_sigma_fit)


popt_BG, _ = curve_fit(['energy'], hem_fit_function_fixed_resolution_scipp, data_grouped_for_fit_150K, 
p0 = {'energy_offset': 0 * sc.Unit('meV'),
    'el_area': 75 * sc.Unit('meV'),
    'lorz1_area': 50 * sc.Unit('meV'),
    'lorz1_HWHM': 0.2 * sc.Unit('meV'),
    'BG': 0.05 * sc.Unit('dimensionless')}
)

popt_BG

DataGroup(sizes={}, keys=[
    energy_offset: DataArray({}),
    res_gauss1_sigma: DataArray({}),
    BG: DataArray({}),
    el_area: DataArray({}),
    lorz1_HWHM: DataArray({}),
    lorz1_area: DataArray({}),
])

In [8]:
popt['res_gauss1_sigma']


<scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64            [meV]  ()  0.018569  1.16929e-08